In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
import multiprocessing

base_url = "https://www.nytimes.com/books/best-sellers/"
current_date = datetime.datetime.now()
best_sellers_data = {}

def scrape_week_data(week_date):
    target_date = week_date
    formatted_date = target_date.strftime("%Y/%m/%d")
    url = f"{base_url}{formatted_date}/"
    response = requests.get(url)

    if response.status_code == 200:
       
        soup = BeautifulSoup(response.content, "html.parser")
        shelf = soup.find("div", itemtype="http://schema.org/ItemList")
        sections = shelf.find_all("section")
        week_data = []

        for section in sections:
            category = section.find("h2").text
            books = section.find_all("li", itemtype="https://schema.org/Book")
            category_books = []
            for book in books:
                title = book.find("h3", itemprop="name").text
                author = book.find("p", itemprop="author").text
                description = book.find("p", itemprop="description").text
                image_url = book.find("img", itemprop="image")["src"]

                category_books.append({
                    "Title": title,
                    "Author": author,
                    "Description": description,
                    "Image_URL": image_url
                })
            week_data.append({
                "Category": category,
                "Books": category_books})

        best_sellers_data[formatted_date] = week_data


target_dates = [current_date - datetime.timedelta(weeks=i) for i in range(104)]

# Create a pool of worker processes
num_processes = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=num_processes)

# Use multiprocessing to scrape the data for each week
pool.map(scrape_week_data, target_dates)

# Print the collected data
for week, data in best_sellers_data.items():
    print(f"Week: {week}")
    for category_data in data:
        print(f"Category: {category_data['Category']}")
        for position, book in enumerate(category_data['Books'], start=1):
            print(f"Position: {position}")
            print(f"Title: {book['Title']}")
            print(f"Description: {book['Description']}")
            print(f"Image URL: {book['Image_URL']}")
            print("\n")

# Close the multiprocessing pool
pool.close()
pool.join()
